In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import tensorflow_datasets as tfds

In [18]:
# load the dataset
(test_set_raw, valid_set_raw, train_set_raw), info = tfds.load(
    'tf_flowers',
    split=['train[:10%]', 'train[10%:25%]', 'train[25%:]'],
    as_supervised=True,
    with_info=True
)


# dataset, info = tfds.load('tf_flowers',
#                           as_supervised=True,
#                           with_info=True)
dataset_size = info.splits['train'].num_examples
class_names = info.features['label'].names
n_class = len(class_names)

In [19]:
# preprocess the images
def preprocess(image, label):
    resized_image = tf.image.resize(image, [244, 244])
    final_image = tf.keras.applications.xception.preprocess_input(
        resized_image)
    return final_image, label

In [20]:
# apply the function, shuffle, and add batching & prefetching
batch_size = 32
train_set = train_set_raw.shuffle(1000)
train_set = train_set.map(preprocess).batch(batch_size).prefetch(1)
val_set = valid_set_raw.map(preprocess).batch(batch_size).prefetch(1)
test_set = test_set_raw.map(preprocess).batch(batch_size).prefetch(1)

In [22]:
# load Xception model, exclude the top (near output) layers
base_model = tf.keras.applications.xception.Xception(
    weights='imagenet',
    include_top=False
)
avg_pool = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output = tf.keras.layers.Dense(n_class, activation='softmax')(avg_pool)
model = tf.keras.Model(inputs=base_model.input, outputs=output)

In [23]:
# freeze the weights of the pretrained layers
for layer in base_model.layers:
    layer.trainable = False

In [24]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, None, None, 3 864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, None, None, 3 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, None, None, 3 0           block1_conv1_bn[0][0]            
______________________________________________________________________________________________

In [25]:
# compile and train
optimizer = tf.keras.optimizers.SGD(lr=0.2, momentum=0.9, decay=0.01)
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy']
)

history = model.fit(train_set, epochs=5, validation_data=val_set)

/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/5


2021-11-03 15:56:11.639096: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


86/86 [==============================] - ETA: 0s - loss: 0.7955 - accuracy: 0.8212

2021-11-03 15:57:01.155569: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


86/86 [==============================] - 60s 682ms/step - loss: 0.7955 - accuracy: 0.8212 - val_loss: 0.6151 - val_accuracy: 0.8512
Epoch 2/5
86/86 [==============================] - 58s 678ms/step - loss: 0.2507 - accuracy: 0.9193 - val_loss: 0.6244 - val_accuracy: 0.8566
Epoch 3/5
86/86 [==============================] - 58s 674ms/step - loss: 0.1246 - accuracy: 0.9582 - val_loss: 0.6117 - val_accuracy: 0.8494
Epoch 4/5
86/86 [==============================] - 60s 697ms/step - loss: 0.0681 - accuracy: 0.9775 - val_loss: 0.5378 - val_accuracy: 0.8711
Epoch 5/5
86/86 [==============================] - 61s 714ms/step - loss: 0.0495 - accuracy: 0.9855 - val_loss: 0.5415 - val_accuracy: 0.8639


In [26]:
# unfreeze the layers
for layer in base_model.layers:
    layer.trainable = True
    
optimizer = tf.keras.optimizers.SGD(lr=0.001, momentum=0.9, decay=0.001)
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy']
)

history = model.fit(train_set, epochs=5, validation_data=val_set)

/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/5


2021-11-03 16:07:28.438243: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


86/86 [==============================] - ETA: 0s - loss: 0.3913 - accuracy: 0.8765

2021-11-03 16:11:38.721638: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


86/86 [==============================] - 262s 3s/step - loss: 0.3913 - accuracy: 0.8765 - val_loss: 0.5232 - val_accuracy: 0.8603
Epoch 2/5
83/86 [===========================>..] - ETA: 11s - loss: 0.0902 - accuracy: 0.9691

KeyboardInterrupt: 

## Object Detection

In [27]:
# adjust the dataset to have fake bounding boxes
def add_random_bounding_boxes(images, labels):
    fake_boxes = tf.random.uniform([tf.shape(images)[0], 4])
    return images, (labels, fake_boxes)

fake_train_set = train_set.take(5).repeat(2).map(add_random_bounding_boxes)

In [33]:
# define the model with Xception
base_model = tf.keras.applications.xception.Xception(
    weights='imagenet',
    include_top=False
)
avg_pool = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
class_output = tf.keras.layers.Dense(n_class, activation='softmax')(avg_pool)
log_output = tf.keras.layers.Dense(4)(avg_pool)
model = tf.keras.Model(inputs=base_model.input,
                       outputs=[class_output, log_output])

In [34]:
model.compile(
    loss=['sparse_categorical_crossentropy', 'mse'],
    loss_weights=[0.8, 0.2],
    optimizer=optimizer, metrics=['accuracy']
)

model.fit(fake_train_set, steps_per_epoch=5, epochs=2)

Epoch 1/2


2021-11-04 09:22:31.025340: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


5/5 [==============================] - 20s 3s/step - loss: 1.4146 - dense_5_loss: 1.6664 - dense_6_loss: 0.4074 - dense_5_accuracy: 0.1813 - dense_6_accuracy: 0.2313
Epoch 2/2
5/5 [==============================] - 16s 3s/step - loss: 1.3340 - dense_5_loss: 1.5698 - dense_6_loss: 0.3909 - dense_5_accuracy: 0.3187 - dense_6_accuracy: 0.2625
